In [2]:
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level
import pickle

set_log_level("ERROR")

In [63]:
df = pd.read_csv('https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/daily-visits.csv')
df.ds = pd.to_datetime(df.ds)
df = df.sort_values(by='ds').reset_index(drop=True)
df

,ds,y
0,2012-05-01,211
1,2012-05-02,201
2,2012-05-03,204
3,2012-05-04,210
4,2012-05-05,180
...,...,...
3688,2022-06-07,276
3689,2022-06-08,278
3690,2022-06-09,284
3691,2022-06-10,253


In [64]:
weather = pd.read_csv('https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/daily-weather.csv')
weather = weather.rename(columns={'datetime':'ds'})
weather.ds = pd.to_datetime(weather.ds)
weather = weather[['ds','tempmax','tempmin','temp','feelslikemax','feelslikemin','feelslike','dew','humidity', 'precip','precipcover','snow','snowdepth','windgust','windspeed','sealevelpressure','cloudcover','visibility','solarradiation', 'solarenergy', 'uvindex','moonphase']]
weather

,ds,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,...,snowdepth,windgust,windspeed,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,moonphase
0,2020-01-01,1.1,-0.8,0.2,-0.6,-3.6,-1.9,-1.6,87.8,2.85,...,0.0,42.5,7.9,999.8,0.8,332.7,33.2,2.8,2,0.17
1,2020-01-02,2.6,-1.0,0.9,2.2,-2.5,-0.5,-2.4,78.7,0.00,...,0.0,31.7,7.7,1005.9,0.8,337.0,27.4,2.4,1,0.22
2,2020-01-03,5.8,2.8,4.8,5.0,2.6,4.0,0.7,74.9,0.00,...,0.0,NaN,7.6,1005.3,0.8,336.7,25.8,2.3,1,0.26
3,2020-01-04,3.2,-1.9,0.5,3.2,-5.8,-1.7,-2.9,78.8,1.22,...,0.0,NaN,11.2,1006.7,0.8,335.4,26.5,2.2,1,0.31
4,2020-01-05,-1.8,-9.9,-5.3,-4.0,-11.8,-8.1,-10.1,69.1,0.26,...,1.9,33.5,13.1,1009.7,0.7,338.1,53.4,4.6,3,0.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903,2022-06-22,27.2,19.0,23.6,28.6,19.0,23.9,20.3,82.3,2.70,...,0.0,42.1,30.2,1007.2,42.7,22.0,330.9,28.6,9,0.82
904,2022-06-23,21.8,12.6,16.8,21.8,12.6,16.8,10.5,67.1,0.00,...,0.0,27.7,15.5,1013.8,44.9,24.1,326.8,28.5,9,0.87
905,2022-06-24,22.9,11.9,17.6,22.9,11.9,17.6,9.9,61.2,0.00,...,0.0,9.7,9.7,1014.3,59.0,24.1,326.9,28.0,9,0.91
906,2022-06-25,21.4,12.8,16.1,21.4,12.8,16.1,12.8,81.3,6.30,...,0.0,46.4,19.4,1009.8,89.9,22.0,171.3,14.9,6,0.95


In [65]:
df = df.merge(weather, on='ds')
df

,ds,y,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,snowdepth,windgust,windspeed,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,moonphase
0,2020-01-01,208,1.1,-0.8,0.2,-0.6,-3.6,-1.9,-1.6,87.8,...,0.0,42.5,7.9,999.8,0.8,332.7,33.2,2.8,2,0.17
1,2020-01-02,332,2.6,-1.0,0.9,2.2,-2.5,-0.5,-2.4,78.7,...,0.0,31.7,7.7,1005.9,0.8,337.0,27.4,2.4,1,0.22
2,2020-01-03,332,5.8,2.8,4.8,5.0,2.6,4.0,0.7,74.9,...,0.0,NaN,7.6,1005.3,0.8,336.7,25.8,2.3,1,0.26
3,2020-01-04,253,3.2,-1.9,0.5,3.2,-5.8,-1.7,-2.9,78.8,...,0.0,NaN,11.2,1006.7,0.8,335.4,26.5,2.2,1,0.31
4,2020-01-05,253,-1.8,-9.9,-5.3,-4.0,-11.8,-8.1,-10.1,69.1,...,1.9,33.5,13.1,1009.7,0.7,338.1,53.4,4.6,3,0.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,2022-06-07,276,25.0,17.0,20.1,25.0,17.0,20.1,11.9,61.2,...,0.0,59.4,9.7,1010.8,25.4,258.3,123.0,10.5,7,0.24
888,2022-06-08,278,23.2,16.0,18.6,23.2,16.0,18.6,12.9,71.8,...,0.0,46.4,14.0,1008.3,64.7,23.5,150.5,12.9,8,0.29
889,2022-06-09,284,17.6,13.8,15.2,17.6,13.8,15.2,11.4,78.4,...,0.0,37.4,14.4,1006.0,76.5,17.9,49.4,4.3,2,0.35
890,2022-06-10,253,19.8,13.8,16.7,19.8,13.8,16.7,12.4,76.6,...,0.0,38.9,9.6,1005.3,43.0,22.6,161.8,14.1,7,0.39


In [66]:
regressors = df.columns.tolist()
regressors.remove('y')
regressors.remove('ds')
regressors

['tempmax',
 'tempmin',
 'temp',
 'feelslikemax',
 'feelslikemin',
 'feelslike',
 'dew',
 'humidity',
 'precip',
 'precipcover',
 'snow',
 'snowdepth',
 'windgust',
 'windspeed',
 'sealevelpressure',
 'cloudcover',
 'visibility',
 'solarradiation',
 'solarenergy',
 'uvindex',
 'moonphase']

In [67]:
m = NeuralProphet(
  yearly_seasonality=False,
  weekly_seasonality=True,
  daily_seasonality=False,
  n_lags=4*7+1,
  n_forecasts=8,
  changepoints_range=0.95,
  n_changepoints=50,
  num_hidden_layers=4,
  d_hidden=36,
  learning_rate=0.005,
).add_country_holidays("CA")

for reg in regressors:
  m = m.add_future_regressor(name=reg)

metrics = m.fit(df, 
                freq='D', 
                # progress='plot'
                )
print(metrics.tail(1))

with open('../models/daily_visits_forecast_model_with_weather.pkl', "wb") as f:
    pickle.dump(m, f)


WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neuralprophet/utils.py:251: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  additive_events_dims = additive_events_dims.append(

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neuralprophet/utils.py:251: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  additive_events_dims = additive_events_dims.append(

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neuralprophet/utils.py:251: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  additive_events_dims = additive_events_dims.append(

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neur

     SmoothL1Loss        MAE       RMSE  RegLoss
203       0.00212  10.001486  12.917857      0.0


In [2]:
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level
import pickle

set_log_level("ERROR")

loaded_model = pickle.load(open('../models/daily_visits_forecast_model_with_weather.pkl', 'rb'))

df = pd.read_csv('https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/daily-visits.csv')
df.ds = pd.to_datetime(df.ds)
df = df.sort_values(by='ds').reset_index(drop=True)

last_timestamp = df.loc[len(df)-1].ds


# df.rename(columns = {target_column:'y', 'Adm. requests cum':"Adm requests cum", 'Pts.waiting for admission CUM':'Pts waiting for admission CUM'}, inplace = True)
# df.dropna(inplace=True)
# regressors = df.columns.tolist()
# regressors.remove('y')
# regressors.remove('ds')

weather = pd.read_csv('https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/daily-weather.csv')
weather = weather.rename(columns={'datetime':'ds'})
weather.ds = pd.to_datetime(weather.ds)
weather = weather[['ds','tempmax','tempmin','temp','feelslikemax','feelslikemin','feelslike','dew','humidity', 'precip','precipcover','snow','snowdepth','windgust','windspeed','sealevelpressure','cloudcover','visibility','solarradiation', 'solarenergy', 'uvindex','moonphase']]

# last_timestamp = df.loc[len(df)-1].ds

df = df.merge(weather, on='ds')

future = pd.concat([df.tail(4*7+1),weather[weather.ds>last_timestamp].head(8)])
future

,ds,y,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,snowdepth,windgust,windspeed,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,moonphase
872,2022-05-23,242.0,15.7,7.3,11.9,15.7,5.6,11.4,1.8,50.7,...,0.0,31.7,14.9,1025.7,0.3,303.8,263.5,22.7,9,0.79
873,2022-05-24,279.0,17.4,8.9,14.0,17.4,8.9,14.0,1.8,44.9,...,0.0,13.7,9.3,1027.3,0.0,344.0,288.0,25.0,8,0.85
874,2022-05-25,261.0,22.8,10.6,17.2,22.8,10.6,17.2,2.9,40.0,...,0.0,15.5,9.4,1026.5,0.0,303.5,313.3,27.3,9,0.89
875,2022-05-26,240.0,22.4,15.7,19.2,22.4,15.7,19.2,9.2,53.6,...,0.0,57.6,11.4,1016.9,0.7,341.9,143.3,12.4,10,0.93
876,2022-05-27,251.0,21.0,16.4,18.9,21.0,16.4,18.9,16.4,86.3,...,0.0,44.6,7.7,1006.8,0.7,336.3,78.4,6.7,3,0.96
877,2022-05-28,228.0,20.0,16.5,18.1,20.0,16.5,18.1,12.7,72.8,...,0.0,35.3,14.8,1005.5,0.6,301.5,194.2,16.8,8,0.99
878,2022-05-29,222.0,25.1,14.3,20.2,25.1,14.3,20.2,10.1,54.5,...,0.0,38.9,13.0,1014.6,0.1,339.9,323.1,27.8,9,1.00
879,2022-05-30,317.0,26.6,16.5,21.5,26.6,16.5,21.5,13.2,59.8,...,0.0,33.1,9.5,1014.5,0.5,339.7,172.7,14.8,9,0.00
880,2022-05-31,290.0,23.9,12.2,16.3,23.9,12.2,16.3,9.0,63.5,...,0.0,37.8,14.9,1016.0,0.8,340.3,183.4,15.8,7,0.00
881,2022-06-01,242.0,17.0,13.0,15.2,17.0,13.0,15.2,9.0,67.2,...,0.0,40.0,11.3,1012.8,0.8,339.1,95.8,8.2,4,0.02


In [3]:

# future = loaded_model.make_future_dataframe(df, periods=12) # periods=m.n_forecasts, n_historic_predictions=False

forecast = loaded_model.predict(future, raw=True, decompose=False)

start = forecast.values.tolist()[0][0]
forecast_length = len(forecast.values.tolist()[0][1:])

forecast_output = pd.DataFrame(columns=['ds','visits','timestamp'])
forecast_output['ds'] = pd.date_range(start=start, periods=forecast_length, freq='D')
forecast_output['visits'] = forecast.values.tolist()[0][1:]
forecast_output['timestamp'] = pd.Timestamp.now().round('S').replace(tzinfo=None)
forecast_output.to_csv('../forecasts/daily_visits_with_weather.csv', index=False)
forecast_output

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neuralprophet/forecaster.py:1649: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_end_to_append)



,ds,visits,timestamp
0,2022-06-21,279.710083,2022-06-22 02:22:41
1,2022-06-22,275.607605,2022-06-22 02:22:41
2,2022-06-23,274.156067,2022-06-22 02:22:41
3,2022-06-24,262.975555,2022-06-22 02:22:41
4,2022-06-25,208.307068,2022-06-22 02:22:41
5,2022-06-26,231.961823,2022-06-22 02:22:41
6,2022-06-27,292.520538,2022-06-22 02:22:41
7,2022-06-28,250.712769,2022-06-22 02:22:41


In [5]:
forecast = loaded_model.predict(future, raw=True, decompose=True)
forecast

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neuralprophet/forecaster.py:1649: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_end_to_append)



,ds,step0,step1,step2,step3,step4,step5,step6,step7,trend0,...,future_regressor_windgust6,future_regressor_windgust7,future_regressor_windspeed0,future_regressor_windspeed1,future_regressor_windspeed2,future_regressor_windspeed3,future_regressor_windspeed4,future_regressor_windspeed5,future_regressor_windspeed6,future_regressor_windspeed7
0,2022-06-21,279.710083,275.607605,274.156067,262.975555,208.307068,231.961823,292.520538,250.712769,19.094131,...,-5.880234,-7.175879,0.090436,0.416005,0.318334,0.227898,0.312908,0.110332,0.365361,0.383448


In [6]:
forecast = forecast.set_index('ds')
forecast

,step0,step1,step2,step3,step4,step5,step6,step7,trend0,trend1,...,future_regressor_windgust6,future_regressor_windgust7,future_regressor_windspeed0,future_regressor_windspeed1,future_regressor_windspeed2,future_regressor_windspeed3,future_regressor_windspeed4,future_regressor_windspeed5,future_regressor_windspeed6,future_regressor_windspeed7
ds,,,,,,,,,,,,,,,,,,,,,
2022-06-21,279.710083,275.607605,274.156067,262.975555,208.307068,231.961823,292.520538,250.712769,19.094131,18.864853,...,-5.880234,-7.175879,0.090436,0.416005,0.318334,0.227898,0.312908,0.110332,0.365361,0.383448


In [8]:
unique_columns = list(set([column[:-1] for column in forecast.columns.tolist()]))
unique_columns

['event_Canada Day',
 "event_New Year's Day (Observed)",
 'future_regressor_tempmin',
 'future_regressor_feelslikemax',
 'future_regressor_snowdepth',
 'future_regressor_sealevelpressure',
 'event_Boxing Day (Observed)',
 'event_Victoria Day',
 'future_regressor_dew',
 'future_regressor_moonphase',
 'future_regressor_windgust',
 'events_additive',
 'future_regressor_precipcover',
 'future_regressor_windspeed',
 'future_regressor_tempmax',
 'event_Christmas Day',
 'future_regressors_additive',
 'event_Christmas Day (Observed)',
 'future_regressor_precip',
 'event_Labour Day',
 'future_regressor_snow',
 'event_Thanksgiving',
 'event_Good Friday',
 'ar',
 'future_regressor_cloudcover',
 'future_regressor_temp',
 'season_weekly',
 'future_regressor_solarradiation',
 'event_Civic Holiday',
 'season_yearly',
 'future_regressor_feelslikemin',
 'future_regressor_visibility',
 'trend',
 'future_regressor_uvindex',
 'event_Family Day',
 'future_regressor_feelslike',
 'future_regressor_solarenerg

In [9]:
for step in range(0,1):
    print(forecast[[column+str(step) for column in unique_columns]].transpose().sort_values(by=0, ascending=False))

KeyError: 0

In [13]:
for step in range(0,8):
    print(step)
    

0
1
2
3
4
5
6
7
